In [1]:
for p in ("Knet", "Plots", "NBInclude")
    Pkg.installed(p) == nothing && Pkg.add(p);
end

In [2]:
using Knet, Plots, NBInclude;
nbinclude("deepppiutils.ipynb"); # loads trnper, devper, tstper, featuresDict, concatAB, train!, trainSep!, modelevaluation 
                                 # ygold, winit, minibatchi, predict, predictSep,loss,zeroone, report, lossgradient, params,
                                 # loadmodel

In [3]:
lossSep(w,x,ygold, predictSep) = nll(predictSep(w,x),ygold);
lossSep(w, data, predictSep) = mean(loss(w,x,y,predictSep) for (x,y) in data);
zerooneSep(w,data,predictSep) = 1 - accuracyi(w,data,predictSep);

In [12]:
# the number of hidden units in the hidden layers of the DeepPPI-CON model
HIDDENSSEP = Any[NOINPUTS, 512, 256, 128]; 
HIDDENSMER = Any[256, 128, NOOUTPUTS]
BATCHSIZE = 64;
trnper = 0.58;
devper = 0.17;
tstper = 1 - trnper - devper;

In [13]:
wa = winit(HIDDENSSEP...);
wb = winit(HIDDENSSEP...);
wMerged = winit(HIDDENSMER...);
w = vcat(wa, wb, wMerged);
dtrn, ddev, dtst = dividedataset(concatAB, ygold, trnper, devper, tstper; batchsize= BATCHSIZE);

In [14]:
# Training Set
trnacc = 0;
for (x, y) in dtrn
    ypred = predictSep(w,x)
    trnacc += accuracyi(ypred, y) 
end
println(trnacc/length(dtrn));
println(loss(w, dtrn, predictSep));
modelevaluation(w, dtrn, predictSep; p=true);

0.5006769169329073
0.6931464
TP: 10005 , TN: 17 , FP: 30 , FN: 9966
Model evaluation:
Accuracy : 0.5006494155010166
Precision : 0.997010463278823
NPV : 0.0017028949211957433
Sensitivity / Recall : 0.500976415777829
Specifity : 0.3617021199637847
MCC : -0.01329223503962594
F1 : 0.6668666266524406


In [15]:
# dev Set
devacc = 0;
for (x, y) in ddev
    ypred = predictSep(w,x)
    devacc += accuracyi(ypred, y) 
end
println(devacc/length(ddev))
println(loss(w, ddev, predictSep))
modelevaluation(w, ddev, predictSep; p=true);

0.4978847579051383
0.6931471
TP: 2917 , TN: 5 , FP: 6 , FN: 2940
Model evaluation:
Accuracy : 0.4979550101400894
Precision : 0.9979473140615986
NPV : 0.001697792868693449
Sensitivity / Recall : 0.49803653739149106
Specifity : 0.4545454132231443
MCC : -0.004102231855258467
F1 : 0.6644646924072364


In [16]:
# Test Set
tstacc = 0;
for (x, y) in dtst
    ypred = predictSep(w,x)
    tstacc += accuracyi(ypred, y) 
end
println(tstacc/length(dtst))
println(loss(w, dtst, predictSep))
modelevaluation(w, dtst, predictSep; p=true);

0.49865562678062675
0.6931473
TP: 4289 , TN: 12 , FP: 9 , FN: 4318
Model evaluation:
Accuracy : 0.4984932776427337
Precision : 0.9979060025598171
NPV : 0.002771362585965043
Sensitivity / Recall : 0.4983153246777837
Specifity : 0.5714285442176883
MCC : 0.00687328892382538
F1 : 0.664703603203045
